In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
batchsize=8

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=0.2,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 validation_split=0.2)

In [4]:
train_data= train_datagen.flow_from_directory(r"C:\Users\Roshith\OneDrive\Desktop\dataset\drowsiness\drowsiness\Prepared_Data\train",
                          target_size=(80,80),batch_size=batchsize,class_mode="categorical", subset="training")
validation_data= train_datagen.flow_from_directory(r"C:\Users\Roshith\OneDrive\Desktop\dataset\drowsiness\drowsiness\Prepared_Data\train",
                          target_size=(80,80),batch_size=batchsize,class_mode="categorical", subset="validation")                                          

Found 23058 images belonging to 2 classes.
Found 5763 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(r"C:\Users\Roshith\OneDrive\Desktop\dataset\drowsiness\drowsiness\Prepared_Data\test",
                          target_size=(80,80),batch_size=batchsize,class_mode="categorical")


Found 4573 images belonging to 2 classes.


In [6]:
bmodel = MobileNet(include_top=False, weights="imagenet",input_shape=(87,87,3))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation="relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation="sigmoid")(hmodel)

model = Model(inputs=bmodel.input,outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [8]:
checkpoint = ModelCheckpoint(r"C:\Users\Roshith\OneDrive\Desktop\dataset\drowsiness\drowsiness\models\model.h5",monitor="val_loss",save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = "val_loss",patience=7,verbose=3, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(moniter= "val_loss",patience=3,verbose= 3, )
callbacks =[checkpoint,earlystop,learning_rate]

In [9]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              #loss=tf.keras.losses.BinaryCrossentropy(),
             # metrics=[tf.keras.metrics.BinaryAccuracy(),
                       #tf.keras.metrics.FalseNegatives()])
model.fit(train_data,batch_size=batchsize,steps_per_epoch=train_data.samples//batchsize,validation_data=validation_data,validation_steps=validation_data.samples//batchsize,callbacks=callbacks,epochs=3)
#model.fit_generator(train_data,epochs=10,verbose=1,shuffle=True,class_weight=None,sample_weight=None,initial_epoch=0,steps_per_epoch=train_data.samples//batchsize,validation_data=validation_data,validation_steps=validation_data.samples//batchsize,callbacks=callbacks,validation_batch_size=8,validation_freq=1,max_queue_size=10,workers=1,use_multiprocessing=False)
#model.fit(tf.expand_dims(train_data,axis=-1),batch_size=batchsize,epochs=10,verbose=1,validation_data=(test_data))

Epoch 1/3
2882/2882 [==============================] - ETA: 0s - loss: 0.2358 - accuracy: 0.9096
Epoch 1: val_loss improved from inf to 0.29423, saving model to C:\Users\Roshith\OneDrive\Desktop\dataset\drowsiness\drowsiness\models\model.h5
2882/2882 [==============================] - 127s 44ms/step - loss: 0.2358 - accuracy: 0.9096 - val_loss: 0.2942 - val_accuracy: 0.9085 - lr: 0.0010
Epoch 2/3
2881/2882 [============================>.] - ETA: 0s - loss: 0.1859 - accuracy: 0.9277
Epoch 2: val_loss did not improve from 0.29423
2882/2882 [==============================] - 110s 38ms/step - loss: 0.1858 - accuracy: 0.9278 - val_loss: 0.3092 - val_accuracy: 0.8970 - lr: 0.0010
Epoch 3/3
2882/2882 [==============================] - ETA: 0s - loss: 0.1756 - accuracy: 0.9319
Epoch 3: val_loss did not improve from 0.29423
2882/2882 [==============================] - 100s 35ms/step - loss: 0.1756 - accuracy: 0.9319 - val_loss: 0.7417 - val_accuracy: 0.8642 - lr: 0.0010
